# ETL Proyecto individual MLOps
Se presentan datos en formato json, procederemos a extraerlos y convertirlos en DataFrames para transformar los datos y posteriormente ser guardados en formato csv y parquet.

In [2]:
import pandas as pd
import json
import ast
import gzip
import pyarrow as pa
import pyarrow.parquet as pq

### ETL games.json.gz

In [ ]:
filas = []

with gzip.open('steam_games.json.gz', 'rb') as archivo:
    for linea in archivo:
        try:
            objeto_json = json.loads(linea)
            filas.append(objeto_json)
        except json.JSONDecodeError:
            print(f'Error de formato JSON en la línea: {linea}')

#Convertir la lista de objetos JSON en un DataFrame.
df_games = pd.DataFrame(filas)

In [4]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eliminamos valores nulos

In [3]:
df_games.dropna(subset=['id'], inplace=True)

Eliminamos columnas irrelevantes y reseteamos indices

In [4]:
# Elimina columnas irrelevantes.
df_games.drop(['publisher','title', 'url', 'reviews_url', 'early_access', 'specs'], axis=1, inplace = True)

# Resetea indices. 
df_games.reset_index(drop=True, inplace=True)

In [7]:
df_games.head()

,genres,app_name,release_date,tags,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
4,NaN,Log Challenge,NaN,"[Action, Indie, Casual, Sports]",2.99,773570,NaN


Extraemos el año de la columna release_date

In [5]:
# Extrae el año de la columna release_date y lo almacena en la columna año_lanzamiento.
df_games['año_lanzamiento'] = df_games['release_date'].str.extract(r'(\d{4})')

# Elimina la columna release_date.
df_games.drop(columns=['release_date'],inplace=True)

In [9]:
# Verifica los nulos 
df_games.isnull().sum()

genres             3282
app_name              1
tags                162
price              1377
id                    0
developer          3298
año_lanzamiento    2167
dtype: int64

Reemplazamos los nulos por 0

In [6]:
df_games.fillna('0',inplace=True)
df_games.isnull().sum()

genres             0
app_name           0
tags               0
price              0
id                 0
developer          0
año_lanzamiento    0
dtype: int64

Renombramos la columna id por item_id y cambiamos el tipo de dato de la columna año de lanzamiento





In [7]:
# Renombra columna id por item_id.
df_games = df_games.rename(columns={'id': 'item_id'})

# Cambia el tipo de dato a int de la columna año de lanzamiento.
df_games['año_lanzamiento'] = df_games['año_lanzamiento'].astype(int)

Verificar si la columna precio contiene valores de tipo str

In [8]:
precio_str = df_games[df_games['price'].apply(lambda x: isinstance(x, str))]
print(precio_str['price'].value_counts())

price
0                                1377
Free                              905
Free to Play                      520
Free To Play                      462
Free Mod                            4
Free Demo                           3
Play for Free!                      2
Third-party                         2
Play Now                            2
Starting at $499.00                 1
Free Movie                          1
Free to Try                         1
Starting at $449.00                 1
Install Theme                       1
Play the Demo                       1
Free HITMAN™ Holiday Pack           1
Play WARMACHINE: Tactics Demo       1
Install Now                         1
Free to Use                         1
Name: count, dtype: int64


Convertir los valores de la columna 'price' que son strings a cero



In [9]:
df_games.loc[df_games['price'].apply(lambda x: isinstance(x, str)), 'price'] = 0

Guardar el dataframe en formato csv

In [ ]:
games = df_games.to_csv('steam_games.csv',index=False)

In [ ]:
# Lee el archivo CSV en un DataFrame de pandas.
games = pd.read_csv('steam_games.csv')

# Convierte el DataFrame de pandas a una tabla de PyArrow.
table = pa.Table.from_pandas(games)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, 'steam_games.parquet')

### ETL users_items.json.gz

In [10]:
users_items = []

with gzip.open('users_items.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            user_item_line = ast.literal_eval(line)
            users_items.append(user_item_line)
        except (SyntaxError, ValueError) as e:
            print(f'Error evaluating line as literal: {e}')

In [20]:
df_items = pd.DataFrame(users_items)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Creamos una nueva fila para cada elemento de la lista de la columna items

In [21]:
df_items = df_items.explode('items').reset_index()
df_items = df_items.drop(columns='index')

In [22]:
# Desanidar la columna items
df_items = pd.concat([df_items, pd.json_normalize(df_items['items'])], axis=1)
df_items.head()

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks,item_id
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0,NaN
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0,NaN
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0,NaN
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic,0.0,0.0,NaN
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force,0.0,0.0,NaN


Eliminamos columnas irrelevantes

In [23]:
df_items.drop(['user_url','items'], axis=1, inplace=True)

In [24]:
# Verificamos los nulos
df_items.isnull().sum()

user_id                   0
items_count               0
steam_id                  0
item_id               16807
item_name             16806
playtime_forever      16806
playtime_2weeks       16806
item_id             5170014
dtype: int64

Eliminamos los nulos



In [25]:
df_items = df_items.dropna()
df_items.isnull().sum()

user_id             0
items_count         0
steam_id            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
item_id             0
dtype: int64

In [26]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           0 non-null      object 
 1   items_count       0 non-null      int64  
 2   steam_id          0 non-null      object 
 3   item_id           0 non-null      object 
 4   item_name         0 non-null      object 
 5   playtime_forever  0 non-null      float64
 6   playtime_2weeks   0 non-null      float64
 7   item_id           0 non-null      object 
dtypes: float64(2), int64(1), object(5)
memory usage: 0.0+ bytes


In [31]:
len(df_items)

0

In [ ]:
#convertimos a csv
items = df_items.to_csv('users_items.csv',index=False) 

In [ ]:
items = pd.read_csv('users_items.csv')

In [ ]:
items['items_id'] = items['item_id'].astype(int)

In [ ]:
# Leer el archivo CSV en un DataFrame de pandas
items = pd.read_csv('users_items.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(items)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, './data/users_items.parquet')

In [ ]:
# Leer el archivo Parquet en una tabla de PyArrow
table = pq.read_table('./data/users_items.parquet')

# Convertir la tabla de PyArrow a un DataFrame de pandas
items_parquet = table.to_pandas()

# Imprimir el DataFrame
items_parquet

Reemplazamos las '' por vacío y cambiamos el tipo de dato de la columna items_id

In [ ]:
df_items['items_id'] = df_items['item_id'].astype(int)

In [ ]:
df_items.to_csv('')

### ETL user_reviews.json.gz

In [32]:
users_reviews = []

with gzip.open('user_reviews.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            user_item = ast.literal_eval(line)
            users_reviews.append(user_item)
        except (SyntaxError, ValueError) as e:
            print(f'Error evaluating line as literal: {e}')

In [33]:
df_reviews = pd.DataFrame(users_reviews)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Hacemos explode en la columa reviews

In [34]:
df_reviews = df_reviews.explode('reviews').reset_index()
df_reviews = df_reviews.drop(columns='index')
df_reviews = pd.concat([df_reviews, pd.json_normalize(df_reviews['reviews'])], axis=1)
df_reviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Observamos los nombres de las columnas su tipo de dato y los no nulos.



In [35]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


In [36]:
nulos3 = df_reviews.isna().sum()
nulos3

user_id         0
user_url        0
reviews        28
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

Utilizar una expresión regular para extraer el año


In [37]:
df_reviews['año'] = df_reviews['posted'].str.extract(r'(\d{4})')
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,año
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011


In [38]:
df_reviews['año'].isna().value_counts()

año
False    49186
True     10147
Name: count, dtype: int64

In [39]:
df_reviews.isnull().sum()

user_id            0
user_url           0
reviews           28
funny             28
posted            28
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
año            10147
dtype: int64

Eliminamos los valores vacíos de la columna año



In [40]:
df_reviews = df_reviews.dropna(subset=['año'])
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,año
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011


In [41]:
df_reviews.isnull().sum()

user_id        0
user_url       0
reviews        0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
año            0
dtype: int64

Realizamos análisis de sentimientos



In [42]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(review):
    
    if isinstance(review, str):
        sentiment = analyzer.polarity_scores(review)

        if sentiment['compound'] >= 0.05:
            return 2
        elif sentiment['compound'] <= -0.05:
            return 0
        else:
            return 1
    else:
        return 1
        
#Aplica la función de análisis de sentimiento a la columna 'reviews' si existe al menos una reseña

df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)

In [43]:
df_reviews['sentiment_analysis'].value_counts()

sentiment_analysis
2    33422
1     9080
0     6684
Name: count, dtype: int64

In [44]:
df_reviews.drop(['reviews','user_url', 'funny', 'last_edited','posted','review'], axis=1, inplace=True)

In [ ]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49186 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49186 non-null  object
 1   item_id             49186 non-null  object
 2   helpful             49186 non-null  object
 3   recommend           49186 non-null  object
 4   año                 49186 non-null  object
 5   sentiment_analysis  49186 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 2.6+ MB


In [ ]:
#convertir a csv
reviews= df_reviews.to_csv('users_reviews.csv',index=False)

In [ ]:
reviews=pd.read_csv('users_reviews.csv')

In [ ]:
# Leer el archivo CSV en un DataFrame de pandas
reviews = pd.read_csv('users_reviews.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(reviews)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, './data/users_reviews.parquet')

In [ ]:
# Leer el archivo Parquet en una tabla de PyArrow
table = pq.read_table('./data/users_reviews.parquet')

# Convertir la tabla de PyArrow a un DataFrame de pandas
reviews_parquet = table.to_pandas()

# Imprimir el DataFrame
reviews_parquet

machine learning

In [ ]:
# Leer el archivo CSV en un DataFrame de pandas
games_ml = pd.read_csv('steam_games_ml.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(games_ml)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, './data/steam_games_ml.parquet')

### Archivo para endpoint @developer

In [ ]:
df_developer = df_games[['item_id', 'price','developer','año_lanzamiento']]
df_developer.to_parquet('./data/steam_games_developer.parquet')

In [ ]:
df_developer.head(2)

,item_id,price,developer,año_lanzamiento
0,761140,4.99,Kotoshiro,2018
1,643980,0,Secret Level SRL,2018


### Archivo para endpoint @userdata

In [ ]:
merged_reviews_games = df_reviews.merge(df_games[['item_id', 'price']])
merged_reviews_games.drop(columns=['helpful','año','sentiment_analysis'], inplace=True)

In [ ]:
merged_reviews_games.head(2)

,user_id,item_id,recommend,price
0,76561197970982479,1250,True,19.99
1,76561197970982479,22200,True,9.99


In [ ]:
merged_reviews_games.to_parquet('./data/userdata.parquet')

### Archivo para endpoint @UserForGenre

In [ ]:
# Hacemos un join de las tablas para tener las horas y el usuario agrupadas a los items
df_join = df_games.merge(df_items, on='item_id', how='inner')

In [ ]:
df_userforgenre = df_join[['genres','item_id','user_id','playtime_forever','año_lanzamiento']]
df_userforgenre.head(8)

,genres,item_id,user_id,playtime_forever,año_lanzamiento
0,"[Action, Indie, Racing]",282010,UTNerd24,5.0,1997
1,"[Action, Indie, Racing]",282010,I_DID_911_JUST_SAYING,0.0,1997


In [ ]:
#convertir a csv
userforgenre = df_userforgenre.to_csv('userforgenre.csv',index=False)

In [ ]:
# Leer el archivo CSV en un DataFrame de pandas
userforgenre = pd.read_csv('userforgenre.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(userforgenre)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, './data/userforgenre.parquet')

### Archivo para endpoint @best_developer_year

In [46]:
merged_df = pd.merge(df_reviews, df_games, on='item_id')
merged_df.head(2)

,user_id,item_id,helpful,recommend,año,sentiment_analysis,genres,app_name,tags,price,developer,año_lanzamiento
0,76561197970982479,1250,No ratings yet,True,2011,2,[Action],Killing Floor,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009
1,76561197970982479,22200,No ratings yet,True,2011,2,"[Action, Indie]",Zeno Clash,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009


In [ ]:
# Unir los DataFrames
merged_df = df_reviews.join(merged_df.set_index('item_id'), how='inner')
merged_df.head(2)

In [47]:
merged_df = merged_df[['año', 'recommend', 'sentiment_analysis', 'developer']]

In [50]:
merged_df['año'] = pd.to_numeric(merged_df['año'], errors='coerce')

In [51]:
merged_df.to_parquet('./data/best_developer_year.parquet')

### Archivo para endpoint @developer_reviews_analysis

In [52]:
# Filtrar por las columnas necesarias
df_reviews = df_reviews[['user_id', 'item_id', 'sentiment_analysis']]
df_games = df_games[['item_id', 'developer']]

# Fusionar los DataFrames
merged = df_reviews.merge(df_games, on='item_id')
merged.head()

,user_id,item_id,sentiment_analysis,developer
0,76561197970982479,1250,2,Tripwire Interactive
1,76561197970982479,22200,2,ACE Team
2,js41637,227300,2,SCS Software
3,js41637,239030,2,3909
4,evcentric,370360,2,Zachtronics


In [53]:
merged.to_parquet('./data/developer_reviews_analysis.parquet')